In [41]:
import numpy
import matplotlib.pyplot
import sympy
from mpl_toolkits.mplot3d import Axes3D
from sympy import Matrix, Symbol, symbols, solveset, solve, simplify, S, diff, det, erf, log, sqrt, pi, sin, cos, tan, asin, acos, atan2, init_printing, pretty_print

theta1,theta2,theta3,theta4,theta5,theta6 = symbols('theta_1 theta_2 theta_3 theta_4 theta_5 theta_6')
theta = Matrix([theta1,theta2,theta3,theta4,theta5,theta6])

step_size = 0.005
theta_max_step = 0.2

def T(x, y, z):
    T_xyz = Matrix([[1, 0, 0, x],
                [0, 1, 0, y],
                [0, 0, 1, z],
                [0, 0, 0, 1]])
    return T_xyz

def Rx(roll):
    R_x = Matrix([[1, 0, 0, 0],
            [0, cos(roll), -sin(roll), 0],
            [0, sin(roll), cos(roll), 0],
            [0, 0, 0, 1]])
    return R_x  

def Ry(pitch):
    R_y = Matrix([[cos(pitch), 0, sin(pitch), 0],
              [0, 1, 0, 0],
              [-sin(pitch), 0, cos(pitch), 0],
              [0, 0, 0, 1]])
    return R_y

def Rz(yaw):
    R_z = Matrix([[cos(yaw), -sin(yaw), 0, 0],
              [sin(yaw), cos(yaw), 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
    return R_z


def R(roll, pitch, yaw):
  R_x = Matrix([[1, 0, 0, 0],
              [0, cos(roll), -sin(roll), 0],
              [0, sin(roll),  cos(roll), 0],
              [0, 0, 0, 1]])
  R_y = Matrix([[ cos(pitch), 0, sin(pitch), 0],
              [0, 1, 0, 0],
              [-sin(pitch), 0, cos(pitch), 0],
              [0, 0, 0, 1]])
  R_z = Matrix([[cos(yaw),-sin(yaw), 0, 0],
              [sin(yaw), cos(yaw), 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
  return R_z*R_y*R_x

# Define transforms to each joint
T1 = Ry(-pi/2) * T(0.187, 0, 0) * Rx(theta1)
T2 = T1 * T(0.096, 0, 0) * Rz(theta2)
T3 = T2 * T(0.205, 0, 0) * Rz(theta3)
T4 = T3 * T(0.124, 0, 0) * Rx(theta4)
T5 = T4 * T(0.167, 0, 0) * Rz(theta5)
T6 = T5 * T(0.104, 0, 0) # fixed point

# Find joint positions in space
p0 = Matrix([0,0,0,1])
p1 = T1 * p0
p2 = T2 * p0
p3 = T3 * p0
p4 = T4 * p0
p5 = T5 * p0
p6 = T6 * p0
# print('T1=',T1,'\n\nT2=',T2,'\n\nT3=',T3,'\n\nT4=',T4,'\n\nT5=',T5,'\n\nT6=',T6)
# print('p1=',p1,'\n\np2=',p2,'\n\np3=',p3,'\n\np4=',p4,'\n\np5=',p5,'\n\np6=',p6)
p = Matrix([p6[0], p6[1], p6[2]]) # coordinates of arm tip
J = p.jacobian(theta)
#   eva.control_go_to([0, 0, -(r(90)+r(30)), 0, -(r(90)-r(30)), r(10)])

theta_i = Matrix(
    [0, 0, -(r(90)+r(30)), 0, -(r(90)-r(30)), r(10)]
) # initial position θi #theta_i = Matrix([0,0,0,0,0,0]
p_i = p.subs({theta1:theta_i[0], theta2:theta_i[1], theta3:theta_i[2], theta4:theta_i[3], theta5:theta_i[4], theta6:theta_i[5]}).evalf()
# cm
p_f = p_i + Matrix(
    [0.05, 0.05, 0.05]
) # moving the arm down in the z axis by 1cm
dp = p_f - p_i  #  distance between end effector is now and our goal position

print("p_i:", p_i)
dp_threshold = 0.001

# while dp.norm() > dp_threshold:
    
#     dp_step = dp * step_size / dp.norm()
#     print("dp_step: ",dp_step)
#     J_i = J.subs({theta1:theta_i[0], theta2:theta_i[1], theta3:theta_i[2], theta4:theta_i[3], theta5:theta_i[4], theta6:theta_i[5]}).evalf()
#     J_inv = J_i.pinv()
#     dtheta = J_inv * dp_step
#     theta_i = theta_i + numpy.clip(dtheta,-1*theta_max_step,theta_max_step)
#     p_i = p.subs({theta1:theta_i[0], theta2:theta_i[1], theta3:theta_i[2], theta4:theta_i[3], theta5:theta_i[4], theta6:theta_i[5]}).evalf()
#     dp = p_f - p_i
    
#     print("step “,step,”:\n θ: [",theta_i,"]\n p:[",p_i,"]")

p_i: Matrix([[0], [-0.252013392501272], [0.238500000000000]])


In [31]:
import math
def radiusToDegree(degree):
    return degree*math.pi/180

r = radiusToDegree

In [5]:
theta

Matrix([
[theta_1],
[theta_2],
[theta_3],
[theta_4],
[theta_5],
[theta_6]])

In [6]:
T1

Matrix([
[0, -sin(theta_1), -cos(theta_1),     0],
[0,  cos(theta_1), -sin(theta_1),     0],
[1,             0,             0, 0.187],
[0,             0,             0,     1]])

In [7]:
T2

Matrix([
[-sin(theta_1)*sin(theta_2), -sin(theta_1)*cos(theta_2), -cos(theta_1),     0],
[ sin(theta_2)*cos(theta_1),  cos(theta_1)*cos(theta_2), -sin(theta_1),     0],
[              cos(theta_2),              -sin(theta_2),             0, 0.283],
[                         0,                          0,             0,     1]])

In [8]:
T3

Matrix([
[-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2),  sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3), -cos(theta_1), -0.205*sin(theta_1)*sin(theta_2)],
[ sin(theta_2)*cos(theta_1)*cos(theta_3) + sin(theta_3)*cos(theta_1)*cos(theta_2), -sin(theta_2)*sin(theta_3)*cos(theta_1) + cos(theta_1)*cos(theta_2)*cos(theta_3), -sin(theta_1),  0.205*sin(theta_2)*cos(theta_1)],
[                          -sin(theta_2)*sin(theta_3) + cos(theta_2)*cos(theta_3),                           -sin(theta_2)*cos(theta_3) - sin(theta_3)*cos(theta_2),             0,       0.205*cos(theta_2) + 0.283],
[                                                                               0,                                                                                0,             0,                                1]])

In [9]:
T4

Matrix([
[-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2),  (sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1),  -(sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*sin(theta_4) - cos(theta_1)*cos(theta_4), -0.124*sin(theta_1)*sin(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*sin(theta_2) - 0.124*sin(theta_1)*sin(theta_3)*cos(theta_2)],
[ sin(theta_2)*cos(theta_1)*cos(theta_3) + sin(theta_3)*cos(theta_1)*cos(theta_2), (-sin(theta_2)*sin(theta_3)*cos(theta_1) + cos(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_1)*sin(theta_4), -(-sin(theta_2)*sin(theta_3)*cos(theta_1) + cos(theta_1)*cos(theta_2)*cos(theta_3))*sin(theta_4) - sin(theta_1)*cos(theta_4),  0.124*sin(theta_2)*cos(theta_1)*cos(theta_3) + 0.205*sin(theta_2)*cos(theta_1) + 0.124*sin(theta_3)*cos(theta_1)*cos(theta_2)],
[                          -sin(theta_2)*sin(theta_3) + cos(theta

In [10]:
T5

Matrix([
[((sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1))*sin(theta_5) + (-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2))*cos(theta_5), ((sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1))*cos(theta_5) - (-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2))*sin(theta_5),  -(sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*sin(theta_4) - cos(theta_1)*cos(theta_4), -0.291*sin(theta_1)*sin(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*sin(theta_2) - 0.291*sin(theta_1)*sin(theta_3)*cos(theta_2)],
[((-sin(theta_2)*sin(theta_3)*cos(theta_1) + cos(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_1)*sin(theta_4))*sin(theta_5) + (sin(theta_2)*cos(theta_1)*cos(theta_3) + sin(theta_3)*cos(theta_1)*cos(theta_2))*cos(theta_5), ((-sin(theta_2)*sin(the

In [11]:
T6

Matrix([
[((sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1))*sin(theta_5) + (-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2))*cos(theta_5), ((sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1))*cos(theta_5) - (-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2))*sin(theta_5),  -(sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*sin(theta_4) - cos(theta_1)*cos(theta_4), 0.104*((sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1))*sin(theta_5) + 0.104*(-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2))*cos(theta_5) - 0.291*sin(theta_1)*sin(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*sin(theta_2) - 0.291*sin(theta_1)*sin(theta_3)*cos(theta_2)],
[((-sin(thet

In [12]:
p0

Matrix([
[0],
[0],
[0],
[1]])

In [13]:
p1

Matrix([
[    0],
[    0],
[0.187],
[    1]])

In [14]:
p2

Matrix([
[    0],
[    0],
[0.283],
[    1]])

In [15]:
p3

Matrix([
[-0.205*sin(theta_1)*sin(theta_2)],
[ 0.205*sin(theta_2)*cos(theta_1)],
[      0.205*cos(theta_2) + 0.283],
[                               1]])

In [16]:
p4

Matrix([
[-0.124*sin(theta_1)*sin(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*sin(theta_2) - 0.124*sin(theta_1)*sin(theta_3)*cos(theta_2)],
[ 0.124*sin(theta_2)*cos(theta_1)*cos(theta_3) + 0.205*sin(theta_2)*cos(theta_1) + 0.124*sin(theta_3)*cos(theta_1)*cos(theta_2)],
[                               -0.124*sin(theta_2)*sin(theta_3) + 0.124*cos(theta_2)*cos(theta_3) + 0.205*cos(theta_2) + 0.283],
[                                                                                                                             1]])

In [17]:
p5

Matrix([
[-0.291*sin(theta_1)*sin(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*sin(theta_2) - 0.291*sin(theta_1)*sin(theta_3)*cos(theta_2)],
[ 0.291*sin(theta_2)*cos(theta_1)*cos(theta_3) + 0.205*sin(theta_2)*cos(theta_1) + 0.291*sin(theta_3)*cos(theta_1)*cos(theta_2)],
[                               -0.291*sin(theta_2)*sin(theta_3) + 0.291*cos(theta_2)*cos(theta_3) + 0.205*cos(theta_2) + 0.283],
[                                                                                                                             1]])

In [18]:
p6

Matrix([
[0.104*((sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1))*sin(theta_5) + 0.104*(-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2))*cos(theta_5) - 0.291*sin(theta_1)*sin(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*sin(theta_2) - 0.291*sin(theta_1)*sin(theta_3)*cos(theta_2)],
[0.104*((-sin(theta_2)*sin(theta_3)*cos(theta_1) + cos(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_1)*sin(theta_4))*sin(theta_5) + 0.104*(sin(theta_2)*cos(theta_1)*cos(theta_3) + sin(theta_3)*cos(theta_1)*cos(theta_2))*cos(theta_5) + 0.291*sin(theta_2)*cos(theta_1)*cos(theta_3) + 0.205*sin(theta_2)*cos(theta_1) + 0.291*sin(theta_3)*cos(theta_1)*cos(theta_2)],
[                                                                                                                0.104*(-sin(theta_2)*sin(theta_3) + cos(theta_2)*cos(theta_3))*cos(theta_5) + 0.104*(-sin(theta_2)*cos(theta_3) - sin(th

In [19]:
p

Matrix([
[0.104*((sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_4)*cos(theta_1))*sin(theta_5) + 0.104*(-sin(theta_1)*sin(theta_2)*cos(theta_3) - sin(theta_1)*sin(theta_3)*cos(theta_2))*cos(theta_5) - 0.291*sin(theta_1)*sin(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*sin(theta_2) - 0.291*sin(theta_1)*sin(theta_3)*cos(theta_2)],
[0.104*((-sin(theta_2)*sin(theta_3)*cos(theta_1) + cos(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) - sin(theta_1)*sin(theta_4))*sin(theta_5) + 0.104*(sin(theta_2)*cos(theta_1)*cos(theta_3) + sin(theta_3)*cos(theta_1)*cos(theta_2))*cos(theta_5) + 0.291*sin(theta_2)*cos(theta_1)*cos(theta_3) + 0.205*sin(theta_2)*cos(theta_1) + 0.291*sin(theta_3)*cos(theta_1)*cos(theta_2)],
[                                                                                                                0.104*(-sin(theta_2)*sin(theta_3) + cos(theta_2)*cos(theta_3))*cos(theta_5) + 0.104*(-sin(theta_2)*cos(theta_3) - sin(th

In [20]:
J

Matrix([
[0.104*((sin(theta_2)*sin(theta_3)*cos(theta_1) - cos(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_4) + sin(theta_1)*sin(theta_4))*sin(theta_5) + 0.104*(-sin(theta_2)*cos(theta_1)*cos(theta_3) - sin(theta_3)*cos(theta_1)*cos(theta_2))*cos(theta_5) - 0.291*sin(theta_2)*cos(theta_1)*cos(theta_3) - 0.205*sin(theta_2)*cos(theta_1) - 0.291*sin(theta_3)*cos(theta_1)*cos(theta_2),   0.104*(sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_5) + 0.104*(sin(theta_1)*sin(theta_2)*cos(theta_3) + sin(theta_1)*sin(theta_3)*cos(theta_2))*sin(theta_5)*cos(theta_4) + 0.291*sin(theta_1)*sin(theta_2)*sin(theta_3) - 0.291*sin(theta_1)*cos(theta_2)*cos(theta_3) - 0.205*sin(theta_1)*cos(theta_2),   0.104*(sin(theta_1)*sin(theta_2)*sin(theta_3) - sin(theta_1)*cos(theta_2)*cos(theta_3))*cos(theta_5) + 0.104*(sin(theta_1)*sin(theta_2)*cos(theta_3) + sin(theta_1)*sin(theta_3)*cos(theta_2))*sin(theta_5)*cos(theta_4) + 0.291*sin(theta_1)*sin(theta_2)*sin(theta_3)

In [21]:
theta_i

Matrix([
[                      -pi/2],
[ -pi/4 - 0.0202241182046491],
[-0.020729982175498 + 3*pi/4],
[                          0],
[  0.0107338534557398 + pi/2],
[                          0]])

In [22]:
p_i

Matrix([
[0.146039751423629],
[                0],
[0.332957619061124]])

In [23]:
p_f

Matrix([
[0.146043109856758],
[                0],
[0.333956890143242]])

In [24]:
dp

Matrix([
[ 3.35843312915252e-6],
[                   0],
[0.000999271082118058]])

In [42]:
while dp.norm() > dp_threshold:
    
    dp_step = dp * step_size / dp.norm()
    print("dp_step: ",dp_step, "\n")
    J_i = J.subs({theta1:theta_i[0], theta2:theta_i[1], theta3:theta_i[2], theta4:theta_i[3], theta5:theta_i[4], theta6:theta_i[5]}).evalf()
    J_inv = J_i.pinv()
    dtheta = J_inv * dp_step
    theta_i = theta_i + numpy.clip(dtheta,-1*theta_max_step,theta_max_step)
    p_i = p.subs({theta1:theta_i[0], theta2:theta_i[1], theta3:theta_i[2], theta4:theta_i[3], theta5:theta_i[4], theta6:theta_i[5]}).evalf()
    dp = p_f - p_i
    
    print("step,”:\n θ: [",theta_i,"]\n p:[",p_i,"]\n")

dp_step:  Matrix([[0.00288675134594813], [0.00288675134594813], [0.00288675134594813]]) 

step,”:
 θ: [ Matrix([[0.0101573871430516], [0.0204464655610816], [-2.10338681423148], [0.00363013148755109], [-1.06213204119433], [0.174532925199433]]) ]
 p:[ Matrix([[0.00285998846930865], [-0.249094457669402], [0.241354535641631]]) ]

dp_step:  Matrix([[0.00288784314870706], [0.00288423203696281], [0.00288817719425644]]) 

step,”:
 θ: [ Matrix([[0.0204863442258310], [0.0408346829800773], [-2.11206130132246], [0.00738622235566541], [-1.07715020865991], [0.174532925199433]]) ]
 p:[ Matrix([[0.00572058126282294], [-0.246177808746923], [0.244210987929225]]) ]

dp_step:  Matrix([[0.00288904148100711], [0.00288153806343948], [0.00288966740473185]]) 

step,”:
 θ: [ Matrix([[0.0309887527308973], [0.0611517196753521], [-2.12041219843053], [0.0112715436899312], [-1.09224253659880], [0.174532925199433]]) ]
 p:[ Matrix([[0.00858188024091161], [-0.243263591787269], [0.247069473436464]]) ]

dp_step:  Matrix(

step,”:
 θ: [ Matrix([[0.211193236485405], [0.345508453425630], [-2.20172090836689], [0.0872505010271724], [-1.31436963444535], [0.174532925199433]]) ]
 p:[ Matrix([[0.0519416204116318], [-0.200457857489766], [0.290363059089313]]) ]

dp_step:  Matrix([[-0.00312343878395128], [-0.00250235234221140], [-0.00299705904485349]]) 

step,”:
 θ: [ Matrix([[0.197229710988180], [0.329535857925387], [-2.20026648249894], [0.0806554194677113], [-1.30187606267846], [0.174532925199433]]) ]
 p:[ Matrix([[0.0487856740822777], [-0.202915285140716], [0.287352779819278]]) ]

dp_step:  Matrix([[0.00319820379939155], [0.00237533962177493], [0.00302146556140950]]) 

step,”:
 θ: [ Matrix([[0.211225338495488], [0.345412895208551], [-2.20165164731365], [0.0872167460017542], [-1.31412964762882], [0.174532925199433]]) ]
 p:[ Matrix([[0.0519522617219198], [-0.200494572554122], [0.290361398203875]]) ]

dp_step:  Matrix([[-0.00315325133239579], [-0.00245316545842472], [-0.00300649052357329]]) 

step,”:
 θ: [ Matrix([

step,”:
 θ: [ Matrix([[0.196878363844312], [0.330593398458844], [-2.20138988110344], [0.0798602119151480], [-1.30190898293754], [0.174532925199433]]) ]
 p:[ Matrix([[0.0485929115964997], [-0.202729358477713], [0.287258415433805]]) ]

dp_step:  Matrix([[0.00350284914272744], [0.00178234771927630], [0.00309083880053363]]) 

step,”:
 θ: [ Matrix([[0.211586739629554], [0.344296583781513], [-2.20084180320721], [0.0868485685621539], [-1.31160057030164], [0.174532925199433]]) ]
 p:[ Matrix([[0.0520712520798535], [-0.200895820341485], [0.290343433327129]]) ]

dp_step:  Matrix([[-0.00346417493143612], [-0.00186914258175647], [-0.00308314742649040]]) 

step,”:
 θ: [ Matrix([[0.196864804199816], [0.330675426149793], [-2.20146489781101], [0.0798138685945866], [-1.30193588195146], [0.174532925199433]]) ]
 p:[ Matrix([[0.0485822594389511], [-0.202713769081968], [0.287254618680052]]) ]

dp_step:  Matrix([[0.00352175684949595], [0.00173977953940294], [0.00309360563862029]]) 

step,”:
 θ: [ Matrix([[0.

step,”:
 θ: [ Matrix([[0.196841780388988], [0.331774577748727], [-2.20232857892277], [0.0793984874020822], [-1.30255036920998], [0.174532925199433]]) ]
 p:[ Matrix([[0.0484925758712412], [-0.202493979194246], [0.287244949730761]]) ]

dp_step:  Matrix([[0.00373214327940157], [0.00118985649900131], [0.00310730559388709]]) 

step,”:
 θ: [ Matrix([[0.211979638920296], [0.343182753490031], [-2.19994716053787], [0.0866768724149998], [-1.30963924704452], [0.174532925199433]]) ]
 p:[ Matrix([[0.0522083988203159], [-0.201249181449743], [0.290352889467744]]) ]

dp_step:  Matrix([[-0.00370248245434930], [-0.00128123506663560], [-0.00310645463177218]]) 

step,”:
 θ: [ Matrix([[0.196851407359752], [0.331863135388899], [-2.20238787207864], [0.0793811136006738], [-1.30261709344917], [0.174532925199433]]) ]
 p:[ Matrix([[0.0484893289280377], [-0.202475591284727], [0.287247171048101]]) ]

dp_step:  Matrix([[0.00374620671360031], [0.00114617418558757], [0.00310680221373499]]) 

step,”:
 θ: [ Matrix([[0.

step,”:
 θ: [ Matrix([[0.197129670766811], [0.333105388043692], [-2.20309259743425], [0.0793628530454427], [-1.30372421149970], [0.174532925199433]]) ]
 p:[ Matrix([[0.0484965072744288], [-0.202212939186916], [0.287315398887523]]) ]

dp_step:  Matrix([[0.00390624914594424], [0.000518445521627667], [0.00307773160800518]]) 

step,”:
 θ: [ Matrix([[0.212425925572585], [0.341812512320641], [-2.19873429297058], [0.0868101490497199], [-1.30783418490389], [0.174532925199433]]) ]
 p:[ Matrix([[0.0523963734676487], [-0.201637728604277], [0.290400102498249]]) ]

dp_step:  Matrix([[-0.00388863498484471], [-0.000609596038395687], [-0.00308332784903164]]) 

step,”:
 θ: [ Matrix([[0.197163649388238], [0.333209562860504], [-2.20314249607110], [0.0793788179828447], [-1.30382683018170], [0.174532925199433]]) ]
 p:[ Matrix([[0.0485011414467774], [-0.202190804458656], [0.287323804192165]]) ]

dp_step:  Matrix([[0.00391653322462366], [0.000463579317773855], [0.00307341206422948]]) 

step,”:
 θ: [ Matrix([

step,”:
 θ: [ Matrix([[0.212751027608510], [0.339866434261861], [-2.19703980244742], [0.0872231845415554], [-1.30562933749334], [0.174532925199433]]) ]
 p:[ Matrix([[0.0526153473838112], [-0.202159904415605], [0.290461934868737]]) ]

dp_step:  Matrix([[-0.00399567851026380], [0.000223838145258014], [-0.00299740716742713]]) 

step,”:
 θ: [ Matrix([[0.197796796472264], [0.334702415124429], [-2.20374301181427], [0.0798572238599724], [-1.30532176951034], [0.174532925199433]]) ]
 p:[ Matrix([[0.0486253200899012], [-0.201880903327547], [0.287477554115317]]) ]

dp_step:  Matrix([[0.00400002202241001], [-0.000385514917863391], [0.00297509698469474]]) 

step,”:
 θ: [ Matrix([[0.212761187555844], [0.339674121847019], [-2.19687914144534], [0.0872640975277993], [-1.30541414698622], [0.174532925199433]]) ]
 p:[ Matrix([[0.0526322514777307], [-0.202211160334318], [0.290466153642303]]) ]

dp_step:  Matrix([[-0.00399862411982153], [0.000300426928821931], [-0.00298676226185147]]) 

step,”:
 θ: [ Matrix

step,”:
 θ: [ Matrix([[0.212448982220632], [0.336740932452717], [-2.19459539979958], [0.0877525743431507], [-1.30202045664248], [0.174532925199433]]) ]
 p:[ Matrix([[0.0527776612262072], [-0.203001495453054], [0.290482477029282]]) ]

dp_step:  Matrix([[-0.00390918121881559], [0.00139062080894492], [-0.00279006737628123]]) 

step,”:
 θ: [ Matrix([[0.198818236174502], [0.336433743591919], [-2.20425118281848], [0.0809143279761922], [-1.30675985933404], [0.174532925199433]]) ]
 p:[ Matrix([[0.0488887054508135], [-0.201563634458868], [0.287710083249121]]) ]

dp_step:  Matrix([[0.00387022617824420], [-0.00156634022083507], [0.00275098666696654]]) 

step,”:
 θ: [ Matrix([[0.212368791568416], [0.336443000262214], [-2.19438050529360], [0.0877843589118405], [-1.30165941233796], [0.174532925199433]]) ]
 p:[ Matrix([[0.0527804531681626], [-0.203083262453414], [0.290479171306373]]) ]

dp_step:  Matrix([[-0.00388691595382491], [0.00149561756087688], [-0.00276676932169961]]) 

step,”:
 θ: [ Matrix([[

step,”:
 θ: [ Matrix([[0.200069412373057], [0.338242232424595], [-2.20471752859221], [0.0824417075452611], [-1.30718618067846], [0.174532925199433]]) ]
 p:[ Matrix([[0.0492642764896423], [-0.201337128005691], [0.287969952206558]]) ]

dp_step:  Matrix([[0.00325201121359940], [-0.00298919321188798], [0.00234289287177767]]) 

step,”:
 θ: [ Matrix([[0.210129265151154], [0.332225837280884], [-2.19168779897254], [0.0877831893066965], [-1.29601140131223], [0.174532925199433]]) ]
 p:[ Matrix([[0.0525454573101258], [-0.204299103409323], [0.290329900458640]]) ]

dp_step:  Matrix([[-0.00328045044183649], [0.00294570304848534], [-0.00235827870464858]]) 

step,”:
 θ: [ Matrix([[0.200161209761721], [0.338378471280136], [-2.20475705395317], [0.0825633457326821], [-1.30714660551771], [0.174532925199433]]) ]
 p:[ Matrix([[0.0492938611794502], [-0.201326557500054], [0.287988120931842]]) ]

dp_step:  Matrix([[0.00318042538073394], [-0.00309348163091698], [0.00230548602181028]]) 

step,”:
 θ: [ Matrix([[0